In [46]:
import ee

In [47]:
ee.Authenticate()

True

In [48]:
ee.Initialize(project='map-environmental-features')


In [49]:
# Coordinates (replace with your target location)
lat, lon = 9.03, 38.74  # Example: Addis Ababa
point = ee.Geometry.Point(lon, lat)

In [50]:
region = point.buffer(500).bounds()  


In [51]:
#  Retrieve Satellite Image (Sentinel-2) ===
image = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(point) \
    .filterDate('2023-01-01', '2023-03-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()


In [52]:
rgb = image.select(['B4', 'B3', 'B2'])

In [53]:
satellite_url = rgb.getThumbURL({
    'region': region,
    'dimensions': 512,
    'format': 'png',
    'min': 0,
    'max': 3000  # Adjust as needed
})

In [54]:
#  Retrieve Elevation Information (SRTM) ===
elevation_img = ee.Image("USGS/SRTMGL1_003")
elevation_sample = elevation_img.sample(region=point, scale=30).first()
elevation_value = elevation_sample.get("elevation").getInfo()
print(f"Elevation: {elevation_value:.2f} meters")

Elevation: 2442.00 meters


In [55]:
# Elevation image thumbnail (for visualization)
elevation_thumb_url = elevation_img.getThumbURL({
    'region': region,
    'dimensions': 1024,
    'min': 2300,
    'max': 2600,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
})
print("Elevation Image URL:", elevation_thumb_url)

Elevation Image URL: https://earthengine.googleapis.com/v1/projects/map-environmental-features/thumbnails/15ee418c9a717928908cb7fff52a2128-26d5a480f52e0672c7fec863ce20f10f:getPixels


In [56]:
#  Image Processing Preprocessing Stub ===
# Resize image to 350x350 (to feed to UGM later)
rgb_resized = rgb.resample('bilinear').reproject(crs='EPSG:4326', scale=10)

In [57]:
rgb_thumb_url = rgb_resized.getThumbURL({
    'region': region,
    'dimensions': 350,
    'format': 'png',
    'min': 0,
    'max': 3000
})
print("Resized Image (350x350) URL:", rgb_thumb_url)

Resized Image (350x350) URL: https://earthengine.googleapis.com/v1/projects/map-environmental-features/thumbnails/ed7aa11a423bc25a213d9250821f78ed-e973b54affa5aa711aa285a72c3de697:getPixels
